# Creating density maps (heatmaps) with DIVAnd
Density maps are created to show the density of observations on a regular grid.
## Packages

In [ ]:
using DIVAnd
using Makie, CairoMakie, GeoMakie
using Statistics
using DelimitedFiles
using LinearAlgebra
using Random
using JupyterFormatter
enable_autoformat()

## Data
The dataset consists of random observations in a rectangular domain.

In [ ]:
NX = 100
NY = 150
# Box size
LX = 100
LY = 2000
# Bounding box
xleft = 100
ybot = -1000
xright = xleft + LX
ytop = ybot + LY
# For the synthetic case, number of points
NP = 200
Random.seed!(1234)
xo = xleft .+ LX * (rand(NP))
xo = xleft .+ LX * (0.5 .+ 0.1 * randn(NP))
yo = ybot .+ LY * rand(NP)
yo = ybot .+ LY * (0.45 .+ 0.1 * randn(NP))
inflation = ones(Float64, NP)
#test for weights on poins
inflation[yo.>ybot+LY/2] .= 1

# Eliminate points out of the box
sel = (xo .> xleft) .& (xo .< xright) .& (yo .> ybot) .& (yo .< ytop)

xo = xo[sel]
yo = yo[sel]
inflation = inflation[sel]

NP = size(xo)[1]

dx = LX / (NX)
dy = LY / (NY)

xg = xleft+dx/2:dx:xleft+LX
yg = ybot+dy/2:dy:ybot+LY
# for pyplot
xp = xleft:dx:xleft+LX
yp = ybot:dy:ybot+LY
maskp, (pmp, pnp), (xip, yip) = DIVAnd.DIVAnd_rectdom(xp, yp)
mask, (pm, pn), (xi, yi) = DIVAnd.DIVAnd_rectdom(xg, yg)

## Perform heatmap computation

In [ ]:
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0)

### Plot observations and field

In [ ]:
fig = Figure()
ax = Axis(
    fig[1, 1],
    aspect = 1,
    title = "Simple heatmap in 2D, automatic uniform scale selection",
)
hm = heatmap!(ax, xip[:, 1], yip[1, :], dens2)
scatter!(ax, xo, yo, color = :white, markersize = 3)
Colorbar(fig[1, 2], hm)
fig

### Adding a wall in the middle

In [ ]:
mask[1:80, 75] .= false

dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0)

fig = Figure()
ax = Axis(fig[1, 1], aspect = 1, title = "Heatmap with a wall")
hm = heatmap!(ax, xip[:, 1], yip[1, :], dens2)
scatter!(ax, xo, yo, color = :white, markersize = 3)
Colorbar(fig[1, 2], hm)
fig

### Wall in the middle and observations one only one side

In [ ]:
sel = (yo .< -10.0)

xo = xo[sel]
yo = yo[sel]
inflation = inflation[sel]
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0)

fig = Figure()
ax = Axis(
    fig[1, 1],
    aspect = 1,
    title = "Adding a wall in the middle and only points to the south",
)
hm = heatmap!(ax, xip[:, 1], yip[1, :], dens2)
scatter!(ax, xo, yo, color = :white, markersize = 3)
Colorbar(fig[1, 2], hm)
fig

### Open wall in the middle

In [ ]:
mask .= true
mask[1:40, 75] .= false
mask[60:end, 75] .= false
dens2, LHM, LCV, LSCV = DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, yo), inflation, 0)

fig = Figure()
ax = Axis(fig[1, 1], aspect = 1, title = "Open wall in the middle")
hm = heatmap!(ax, xip[:, 1], yip[1, :], dens2)
scatter!(ax, xo, yo, color = :white, markersize = 3)
Colorbar(fig[1, 2], hm)
fig

### Open wall in the middle and increased Ly

In [ ]:
dens2, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask, (pm, pn), (xi, yi), (xo, 2 * yo), inflation, (4, 100))

fig = Figure()
ax = Axis(fig[1, 1], aspect = 1, title = "Open wall in the middle and increased Ly")
hm = heatmap!(ax, xip[:, 1], yip[1, :], dens2)
scatter!(ax, xo, yo, color = :white, markersize = 3)
Colorbar(fig[1, 2], hm)
fig

## 1-dimensional case

In [ ]:
NX1D = 500
LX1D = 8
xleft1D = -4
dx1D = LX1D / (NX1D)

xg1D = xleft1D+dx1D/2:dx1D:xleft1D+LX1D
mask1D, pm1D, xi1D = DIVAnd.DIVAnd_rectdom(xg1D)

xo = randn(1000)
inflation = ones(size(xo))

dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0; Ladaptiveiterations = 0)
dens1Db, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask1D,
    pm1D,
    xi1D,
    (xo,),
    inflation,
    0;
    Ladaptiveiterations = 0,
    myheatmapmethod = "GridKernel",
    alphabc = 0,
)
dens1Dc, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask1D,
    pm1D,
    xi1D,
    (xo,),
    inflation,
    0;
    Ladaptiveiterations = 0,
    myheatmapmethod = "GridKernel",
    optimizeheat = false,
    alphabc = 0,
)

gaussiansol = 1 / sqrt(2 * pi) * exp.(-0.5 * (xi1D[1] .^ 2.0));

### Plot density

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Density")
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], gaussiansol)
lines!(ax, xi1D[1], dens1Db)
display(fig)
@show norm(dens1D - dens1Db),
norm(dens1D),
norm(dens1Db - dens1Dc),
norm(gaussiansol - dens1D);

In [ ]:
fig = Figure()
ax = Axis(
    fig[1, 1],
    title = "Difference between GridPoint and DataPoint method.\nDifference with or without optimization",
)
lines!(ax, xi1D[1], dens1D - dens1Db)
lines!(ax, xi1D[1], dens1D - dens1Dc)
lines!(ax, xi1D[1], dens1Db - dens1Dc)
fig

In [ ]:
NX1D = 500
LX1D = 8
xleft1D = -4
dx1D = LX1D / (NX1D)

xg1D = xleft1D+dx1D/2:dx1D:xleft1D+LX1D
mask1D, pm1D, xi1D = DIVAnd.DIVAnd_rectdom(xg1D)

xo = randn(100)
inflation = ones(size(xo))

dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0; Ladaptiveiterations = 0)
@time dens1D1, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 1; Ladaptiveiterations = 1)
@time dens1D1, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 1; Ladaptiveiterations = 2)
@time dens1D1nopt, LHM, LCV, LSCV = 
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 1; Ladaptiveiterations = 2, optimizeheat = false)
@time dens1D1nopt, LHM, LCV, LSCV = 
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 1; Ladaptiveiterations = 2, optimizeheat = false)
)

gaussiansol = 1 / sqrt(2 * pi) * exp.(-0.5 * (xi1D[1] .^ 2.0));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Density")
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], gaussiansol)
lines!(ax, xi1D[1], dens1D1)
display(fig)
@show norm(dens1D1 - dens1D1nopt);

In [ ]:
NSAM = 100
CV1 = zeros(Float64, NSAM)
CV2 = zeros(Float64, NSAM)
BW = zeros(Float64, NSAM)
for jjj = 1:NSAM
    HH = 0.005 * jjj
    dens1D, LHM, LCV, LSCV =
        DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, HH; Ladaptiveiterations = 0)
    @show norm(dens1D - gaussiansol), HH, LCV, LSCV
    CV1[jjj] = LCV
    CV2[jjj] = LSCV
    BW[jjj] = HH

end
dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0.195; Ladaptiveiterations = 0);
gaussiansol = 1 / sqrt(2 * pi) * exp.(-0.5 * (xi1D[1] .^ 2.0));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Density")
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], gaussiansol)
display(fig)

In [ ]:
lines(BW, CV1)

In [ ]:
BW[findmax(CV1)[2]]

In [ ]:
lines(BW, CV2)

In [ ]:
BW[findmin(CV2)[2]]

In [ ]:
findmin(CV2)

In [ ]:
NX1D = 500
LX1D = 20
xleft1D = -10
dx1D = LX1D / (NX1D)

xg1D = xleft1D+dx1D/2:dx1D:xleft1D+LX1D
mask1D, pm1D, xi1D = DIVAnd.DIVAnd_rectdom(xg1D)

xa = randn(100) .+ 4
xb = 2 * randn(100) .- 4.0
xo = [xa..., xb...]

inflation = ones(size(xo))

dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0; Ladaptiveiterations = 0)
dens1D1, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0; Ladaptiveiterations = 4)

gaussiansol =
    0.5 / sqrt(2 * pi) * exp.(-0.5 * ((xi1D[1] .- 4) .^ 2.0)) +
    0.5 / (2 * sqrt(2 * pi)) * exp.(-0.5 * (((xi1D[1] .+ 4) ./ 2) .^ 2.0));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Density")
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], gaussiansol)
lines!(ax, xi1D[1], dens1D1)
display(fig)

In [ ]:
NX1D = 500
LX1D = 40
xleft1D = -20
dx1D = LX1D / (NX1D)

xg1D = xleft1D+dx1D/2:dx1D:xleft1D+LX1D
mask1D, pm1D, xi1D = DIVAnd.DIVAnd_rectdom(xg1D)

xa = randn(100) .+ 10
xb = 2 * randn(100) .- 10.0
xo = [xa..., xb...]

inflation = ones(size(xo))

dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0; Ladaptiveiterations = 0)
dens1D1, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 1; Ladaptiveiterations = 0)
dens1D1i, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 1.3; Ladaptiveiterations = 1)

gaussiansol =
    0.5 / sqrt(2 * pi) * exp.(-0.5 * ((xi1D[1] .- 10) .^ 2.0)) +
    0.5 / (2 * sqrt(2 * pi)) * exp.(-0.5 * (((xi1D[1] .+ 10) ./ 2) .^ 2.0));

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], title = "Density")
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], gaussiansol)
lines!(ax, xi1D[1], dens1D1)
lines!(ax, xi1D[1], dens1D1i)
display(fig)

## Some effects of Boundary Conditions 
(uniform data in 0-1)

In [ ]:
NX1D = 500
LX1D = 1
xleft1D = 0
dx1D = LX1D / (NX1D)

xg1D = xleft1D+dx1D/2:dx1D:xleft1D+LX1D
mask1D, pm1D, xi1D = DIVAnd.DIVAnd_rectdom(xg1D)

xo = rand(1000)

inflation = ones(size(xo))
mask1D[1] = false
#mask1D[end]=false
dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0; Ladaptiveiterations = 0)
dens1D1, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask1D,
    pm1D,
    xi1D,
    (xo,),
    inflation,
    0;
    Ladaptiveiterations = 0,
    alphabc = 0,
)
#dens1D1i= DIVAnd_heatmap(mask1D,pm1D,xi1D,(xo,),inflation,0.5;Ladaptiveiterations=10)
#dens1D[mask1D.==false].=NaN
#dens1D1[mask1D.==false].=NaN

In [ ]:
fig = Figure()
ax = Axis(
    fig[1, 1],
    title = "Density, left closed domain, right with different boundary conditions",
)
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], dens1D1)
display(fig)

In [ ]:
dens1D, LHM, LCV, LSCV =
    DIVAnd_heatmap(mask1D, pm1D, xi1D, (xo,), inflation, 0.3; Ladaptiveiterations = 0)
dens1D1, LHM, LCV, LSCV = DIVAnd_heatmap(
    mask1D,
    pm1D,
    xi1D,
    (xo,),
    inflation,
    0.3;
    Ladaptiveiterations = 0,
    alphabc = 0,
)
#dens1D1i= DIVAnd_heatmap(mask1D,pm1D,xi1D,(xo,),inflation,0.5;Ladaptiveiterations=10)
#dens1D[mask1D.==false].=NaN
#dens1D1[mask1D.==false].=NaN

In [ ]:
fig = Figure()
ax = Axis(
    fig[1, 1],
    title = "Density, left closed domain, right with different boundary conditions, L increased",
)
lines!(ax, xi1D[1], dens1D)
lines!(ax, xi1D[1], dens1D1)
display(fig)